In [394]:
#General imports

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings("ignore")
import random
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import export_graphviz 
import pandas as pd
from math import sqrt

%matplotlib inline
bookingHotels = pd.read_csv('bookingHotelsClean2.csv')



In [395]:
bookingHotels = bookingHotels.groupby(['SnapShotDate','TimeToTravel','LengthOfStay']).filter(lambda x: len(x) == 100)


In [396]:
#bookingHotels.head()

In [397]:
#make the index for the predictions
ind = 1 
for i , row in bookingHotels.iterrows():
    if ind == 101:
        ind = 1 
    bookingHotels.at[i,'index'] = ind
    ind = ind+1

In [398]:
#bookingHotels['index'].describe()

In [399]:
bookingHotels.groupby(['SnapShotDate','TimeToTravel','LengthOfStay']).count()

index  Name  Distance  Rate  Grade  \
SnapShotDate TimeToTravel LengthOfStay                                       
2023-03-19   1.0          1               100   100       100   100    100   
                          2               100   100       100   100    100   
                          3               100   100       100   100    100   
             2.0          2               100   100       100   100    100   
                          3               100   100       100   100    100   
...                                       ...   ...       ...   ...    ...   
2023-03-26   29.0         2               100   100       100   100    100   
                          3               100   100       100   100    100   
             30.0         1               100   100       100   100    100   
                          2               100   100       100   100    100   
                          3               100   100       100   100    100   

                                        Price  Review  TypeRoom  TypeBed  \
SnapShotDate TimeToTravel LengthOfStay                                     
2023-03-19   1.0          1               100     100       100      100   
                          2               100     100       100      100   
                          3               100     100       100      100   
             2.0          2               100     100       100      100   
                          3               100     100       100      100   
...                                       ...     ...       ...      ...   
2023-03-26   29.0         2               100     100       100      100   
                          3               100     100       100      100   
             30.0         1               100     100       100      100   
                          2               100     100       100      100   
                          3               100     100       100      100   

                                        Start_Date  ...  \
SnapShotDate TimeToTravel LengthOfStay              ...   
2023-03-19   1.0          1                    100  ...   
                          2                    100  ...   
                          3                    100  ...   
             2.0          2                    100  ...   
                          3                    100  ...   
...                                            ...  ...   
2023-03-26   29.0         2                    100  ...   
                          3                    100  ...   
             30.0         1                    100  ...   
                          2                    100  ...   
                          3                    100  ...   

                                        Location_Upper East Side  \
SnapShotDate TimeToTravel LengthOfStay                             
2023-03-19   1.0          1                                  100   
                          2                                  100   
                          3                                  100   
             2.0          2                                  100   
                          3                                  100   
...                                                          ...   
2023-03-26   29.0         2                                  100   
                          3                                  100   
             30.0         1                                  100   
                          2                                  100   
                          3                                  100   

                                        Location_Upper West Side  \
SnapShotDate TimeToTravel LengthOfStay                             
2023-03-19   1.0          1                                  100   
                          2                                  100   
                          3                                  100   
             2.0          2                

In [400]:
bookingHotels = bookingHotels.astype({'index':'float'})

In [401]:
#bookingHotels.info()

In [402]:
bookingHotels = bookingHotels[['index', 'Distance', 'Rate', 'Grade', 'Price', 'Review',
       'TypeRoom', 'TypeBed', 'LengthOfStay', 'TimeToTravel',
       'Cancellation', 'Prepayment', 'Price_per_night']]

In [403]:
bookingHotels

,index,Distance,Rate,Grade,Price,Review,TypeRoom,TypeBed,LengthOfStay,TimeToTravel,Cancellation,Prepayment,Price_per_night
0,1.0,1.9,8.5,8,759.0,821,3,2,1,1.0,0,0,759.000000
1,2.0,1.5,8.8,9,715.0,1444,4,3,1,1.0,1,1,715.000000
2,3.0,1.4,8.1,8,400.0,12866,3,2,1,1.0,1,1,400.000000
3,4.0,1.0,8.6,9,687.0,6645,5,3,1,1.0,0,0,687.000000
4,5.0,2.0,7.7,7,639.0,2309,4,3,1,1.0,1,0,639.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
44385,96.0,2.5,7.9,7,4183.0,1036,3,2,3,30.0,1,1,1394.333333
44386,97.0,1.6,8.3,8,3795.0,711,3,2,3,30.0,0,0,1265.000000
44387,98.0,0.7,5.6,6,2885.0,4147,3,1,3,30.0,1,0,961.666667
44388,99.0,5.3,7.9,7,3580.0,2416,3,2,3,30.0,0,0,1193.333333


In [404]:
#the model that predict the index
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error


X = bookingHotels.drop(['index','Grade'], axis =1).copy()
y = bookingHotels['index']

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3, random_state=0)
reg = GradientBoostingRegressor(random_state=0)
reg.fit(X_train, y_train)
y_predicted = reg.predict(X_test)
print("r2: " ,r2_score(y_test , y_predicted))
rms = mean_squared_error(y_test, y_predicted, squared=False)
print("rmse: ", rms)

r2:  0.20543251065319768
rmse:  25.479111601947768


In [406]:
bookingHotels

,index,Distance,Rate,Grade,Price,Review,TypeRoom,TypeBed,LengthOfStay,TimeToTravel,Cancellation,Prepayment,Price_per_night
0,1.0,1.9,8.5,8,759.0,821,3,2,1,1.0,0,0,759.000000
1,2.0,1.5,8.8,9,715.0,1444,4,3,1,1.0,1,1,715.000000
2,3.0,1.4,8.1,8,400.0,12866,3,2,1,1.0,1,1,400.000000
3,4.0,1.0,8.6,9,687.0,6645,5,3,1,1.0,0,0,687.000000
4,5.0,2.0,7.7,7,639.0,2309,4,3,1,1.0,1,0,639.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
44385,96.0,2.5,7.9,7,4183.0,1036,3,2,3,30.0,1,1,1394.333333
44386,97.0,1.6,8.3,8,3795.0,711,3,2,3,30.0,0,0,1265.000000
44387,98.0,0.7,5.6,6,2885.0,4147,3,1,3,30.0,1,0,961.666667
44388,99.0,5.3,7.9,7,3580.0,2416,3,2,3,30.0,0,0,1193.333333


In [407]:
bookingHotels['pred_index'] = bookingHotels.apply(lambda x: reg.predict([x.drop(['index','Grade'],axis =0)])[0],  axis=1)

In [408]:
bookingHotels.head()

,index,Distance,Rate,Grade,Price,Review,TypeRoom,TypeBed,LengthOfStay,TimeToTravel,Cancellation,Prepayment,Price_per_night,pred_index
0,1.0,1.9,8.5,8,759.0,821,3,2,1,1.0,0,0,759.0,54.537842
1,2.0,1.5,8.8,9,715.0,1444,4,3,1,1.0,1,1,715.0,40.468669
2,3.0,1.4,8.1,8,400.0,12866,3,2,1,1.0,1,1,400.0,24.443236
3,4.0,1.0,8.6,9,687.0,6645,5,3,1,1.0,0,0,687.0,47.549438
4,5.0,2.0,7.7,7,639.0,2309,4,3,1,1.0,1,0,639.0,50.397030


In [409]:
#bookingHotels.info()

In [410]:
booking_collection = {} 


In [411]:
#seperate the data frame to 298 part of 100 rows for checking our predictions to the index
j = 100
i = 0
k = 0
while(i<29800):
    booking_collection[k] = bookingHotels[i:j]
    i+=100
    j+=100
    k+=1    

In [412]:
#booking_collection[1]

In [413]:
rmse_list = []

for book in booking_collection:
    mse_list = []
    j = 1
    book = temp.sort_values('pred_index')
    rank = 1
    for i ,row in sort_hotel.iterrows():
        book.at[i,'pred_index'] = rank
        rank =rank +1
        mse_list.append((book.at[i,'index']-book.at[i,'pred_index'])**2)
    rmse_list.append(sqrt(sum(mse_list)/100))
    
    

In [414]:
#rmse_list

In [415]:
#Standard deviation  
sum(rmse_list)/298

33.788755526062225